# Assignment 2: The Winter is here
##### This works best with epic battle music. No spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [3]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
from PIL import Image


%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire

In [4]:
import random
from collections import namedtuple

In [5]:
resnet_input = 224 #size of resnet18 input images

In [125]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
batch_size = 2
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [7]:
classes = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')
counts = [0]*21

In [8]:
classes.index('aeroplane')

1

In [9]:
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    
ra = Rectangle(1, 1, 3, 3)
rb = Rectangle(1, 1, 5, 5)

type(area(ra,rb))

int

In [10]:
def create_hierarchical_dirs():
    if not os.path.exists('data/train'):
        os.makedirs('data/train')      
        
    for i in range(len(classes)):
        dirname = 'data/train/' + classes[i]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

create_hierarchical_dirs()

In [11]:
def jamie_bronn_build_dataset():
    
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')    
    for img_name in dir_names:
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                for xmin in object1.iter('xmin'):
                    x1 = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    x2 = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    y1 = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    y2 = int(float(ymax.text))

                img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
                img = img.crop((x1,y1,x2,y2)).resize((resnet_input,resnet_input))
                if dirname in classes:
                    img.save('data/train/' + dirname + '/' + str(counts[classes.index(dirname)]) + ".jpg");
                    counts[classes.index(dirname)]+=1
                    
    
        
jamie_bronn_build_dataset()

In [12]:
def build_background_class():
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    average = int(sum(counts[1:])/len(counts[1:]))
    average = 761
    counts[0] = 0
    while(counts[0]!=average):        
        print(counts[0])
        select = 1
        img_name = random.choice(dir_names)
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
        imsize = img.size;
        x1 = random.randrange(imsize[0])
        x2 = random.randrange(imsize[0])
        y1 = random.randrange(imsize[1])
        y2 = random.randrange(imsize[1])
        if(x1>x2):
            temp = x1
            x1=x2
            x2=temp
        if(y1>y2):
            temp = y1
            y1=y2
            y2=temp
        print(x1,y1,x2,y2)
        if(x1==x2 or y1==y2):
            continue
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                intersection = 0
                for xmin in object1.iter('xmin'):
                    xa = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    xb = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    ya = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    yb = int(float(ymax.text))
                ra = Rectangle(x1, x2, y1, y2)
                rb = Rectangle(xa, xb, ya, yb)
                if(area(ra,rb)!=None):
                    intersection = area(ra,rb)
                union = ((x2-x1)*(y2-y1))+((xb-xa)*(yb-ya))-intersection
                if(float(intersection)/union>0.5):
                    select = 0

        if(select==1):
            img = img.crop((x1,y1,x2,y2)).resize((resnet_input,resnet_input))
            img.save('data/train/__background__/' + str(counts[0]) + ".jpg");
            counts[0] += 1
    
build_background_class()

0
18 156 53 186
0
81 62 333 266
1
153 125 227 144
2
84 161 493 220
3
364 61 384 318
4
63 172 97 491
5
26 152 161 374
6
46 116 350 356
7
251 55 368 55
7
249 47 333 51
8
177 0 190 187
9
484 114 495 323
10
207 328 246 409
11
337 89 494 261
12
126 266 191 358
13
170 106 327 246
14
101 18 365 107
15
209 118 379 270
16
291 249 432 292
17
45 26 390 340
18
154 109 457 216
19
243 141 467 169
20
273 331 455 372
21
172 154 252 156
22
261 48 292 322
23
117 237 122 322
24
293 92 327 319
25
125 21 365 226
26
215 350 232 404
27
175 75 442 322
28
60 9 177 191
29
281 54 469 346
30
48 199 176 336
31
308 83 372 227
32
40 113 364 302
33
234 328 384 344
34
244 127 276 251
35
223 71 497 113
36
236 250 335 301
37
28 181 365 221
38
91 148 227 313
39
8 9 329 172
40
36 223 224 249
40
50 196 437 346
41
71 34 230 72
42
175 16 212 359
43
205 156 336 237
44
154 167 466 186
45
331 127 402 143
46
61 90 179 306
47
223 133 468 241
48
207 160 324 225
49
230 64 444 105
50
60 231 278 299
51
187 118 459 342
52
379 183 403 

91 23 186 310
438
258 59 485 142
439
63 144 296 186
440
17 244 191 288
441
273 76 339 386
442
377 171 391 221
443
241 160 462 306
444
269 185 451 305
445
27 154 208 374
446
111 64 119 182
447
17 168 456 288
448
356 83 429 317
449
58 172 146 354
450
3 15 450 24
451
109 129 128 199
452
44 19 111 204
453
272 90 337 120
454
75 25 219 48
455
134 248 178 258
456
0 11 289 246
457
194 75 198 285
458
81 36 484 287
459
317 176 418 227
460
27 12 496 247
461
33 256 309 307
462
79 282 155 334
463
33 160 321 261
464
353 59 369 287
465
224 71 287 121
466
79 187 252 359
467
211 115 298 215
468
78 108 134 238
469
47 73 329 332
470
193 261 212 281
471
33 302 140 331
472
86 16 475 63
473
17 109 238 256
474
93 143 482 238
475
172 121 395 270
476
45 22 444 224
477
270 29 410 78
478
17 329 321 481
479
62 100 143 134
480
296 106 390 329
481
153 110 159 119
482
156 56 408 162
483
185 153 334 186
484
396 130 493 304
485
67 250 189 347
486
82 60 261 231
487
127 358 333 365
488
310 368 331 372
489
411 258 467 31

In [130]:
bb_list = []
class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        
        
        if (train==True):            
            dir1 = '/train/'
            dir_names = os.listdir(root_dir + dir1)
            img_names=[]
            labels = []
            count = 0
            for c in dir_names:
                names = os.listdir(root_dir + dir1 + c)
                N = len(names)
                for n in range(N):
                    img_names.append(str(root_dir + dir1 + c + '/' + names[n]))
                    labels += [count]
                count+=1           
            self.labels = labels 
         
        
        
        else:
            
            img_names=[]
            dir1 = '/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages'
            dir_names = os.listdir(root_dir + dir1)
            for name in dir_names:
                bb_sub_list = []
                img_names.append(str(root_dir + dir1 + '/' + name))
                tree = ET.parse('data/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + name[:-4] + '.xml')
                root = tree.getroot()
                for object1 in root.iter('object'):
                    for name1 in object1.iter('name'):
                        class1 = name1.text                        
                    for bndbox in object1.iter('bndbox'):
                        for xmin in object1.iter('xmin'):
                            x1 = int(float(xmin.text))
                        for xmax in object1.iter('xmax'):
                            x2 = int(float(xmax.text))
                        for ymin in object1.iter('ymin'):
                            y1 = int(float(ymin.text))
                        for ymax in object1.iter('ymax'):
                            y2 = int(float(ymax.text))
                        if(classes.__contains__(class1)):
                            bb_sub_list.append([x1,y1,x2,y2,classes.index(class1)])        
                bb_list.append(bb_sub_list)
                #print(bb_sub_list)
            self.bb_list = bb_list 
        
        self.img_names = img_names
        self.train = train
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        # Begin
        
        return len(self.img_names)        
        
    def __getitem__(self, idx):
        # Begin      
        
        
        if(self.train==True):
            img_pil = Image.open(self.img_names[idx])
            return self.transform(img_pil), self.labels[idx]
        else:
            print(self.bb_list[idx])
            return self.img_names[idx] , self.bb_list[idx]       
        

In [131]:
print('as'+str(2))

as2


## Train the network
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [132]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.ToTensor()])
#                                          transforms.RandomHorizontalFlip()])
train_dataset = hound_dataset(root_dir='./data', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = hound_dataset(root_dir='./data', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [137]:
test_dataiter = iter(test_loader)
test_images, test_labels = test_dataiter.next()
bb_list[0:2]

[[48, 240, 195, 371, 12], [8, 12, 352, 498, 15]]
[[139, 200, 207, 301, 19]]


[[[48, 240, 195, 371, 12], [8, 12, 352, 498, 15]], [[139, 200, 207, 301, 19]]]

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
print(type(test_images[0]))
img = Image.open(test_images[0])
for i in range()
box = [100,100,200,200]
img.crop(box)
#imshow(Image.open(test_images[0]))

In [78]:
bb_list = []
bb_list.append([[1,2,3,4,5],[2,3,5,4,7]])
bb_list.append([[1,2,3,4,5],[2,3,5,4,7]])
bb_list[0]

[[1, 2, 3, 4, 5], [2, 3, 5, 4, 7]]

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [141]:
resnet18 = models.resnet18(pretrained=True)

for param in resnet18.parameters():
    param.requires_grad = False

resnet18.fc = nn.Linear(resnet18.fc.in_features, 21)
resnet18.cuda()

# Add code for using CUDA here

RuntimeError: cuda runtime error (30) : unknown error at /opt/conda/conda-bld/pytorch_1502009910772/work/torch/lib/THC/THCGeneral.c:70

In [ ]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(resnet18.fc.parameters(), learning_rate, hyp_momentum)

In [ ]:
def arya_train():
    # Begin
    for epoch in range(num_epochs): 
        resnet18.train()        
        for i, (input1,target) in enumerate(train_loader):
            
            input_var = torch.autograd.Variable(input1).cuda()            
            target = target.cuda(async = True)
            target_var = torch.autograd.Variable(target)
            
            output = resnet18(input_var)            
            loss = criterion(output, target_var)           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = output.float()
            loss = loss.float()
            
            print(str(epoch)+ '---'+str(i) + '--------' +  str(loss) + '-------' +str(output))

In [ ]:
%time arya_train()

# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [ ]:
def theon_sliding_window():
    # Begin
    boxes=[]
    imres = [[192,192],[192,96],[96,192],[128,128],[128,64],[64,128],[64,64],[64,32],[32,64]]
    for i in range(resnet_input):
        for j in range(resnet_input):
            for window in imres:
                boxes.append([i-window[0]/2, j-window[1]/2, i+window[0]/2, j+window[1]/2])
    return np.asarray(np.array(boxes))


"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [ ]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.3):
            
        pick = []

        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)
        
        while len(idxs) > 0:
            
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)
            suppress = [last]
            
            for pos in xrange(0, last):
                
                j = idxs[pos]
               
                xx1 = max(x1[i], x1[j])
                yy1 = max(y1[i], y1[j])
                xx2 = min(x2[i], x2[j])
                yy2 = min(y2[i], y2[j])
               
                w = max(0, xx2 - xx1 + 1)
                h = max(0, yy2 - yy1 + 1)

                overlap = float(w * h) / area[j]

                if overlap > threshold:
                    suppress.append(pos)

            idxs = np.delete(idxs, suppress)

        return boxes[pick]
        

Daenerys, the queen, then orders her army to test out the trained model on the test dataset.

In [139]:
def daenerys_test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model
    total_bbox = 0
    total_correct_bbox = 0
    sliding_windows = theon_sliding_window()
    sliding_windows = aegon_targaryen_non_maximum_supression(sliding_windows)
    for i, (input1,bb_list1) in enumerate(test_loader):
            img_n = -1
            for img_name in input1:
                img_n+=1
                img = Image.open(img_name)
                img = img.resize((resnet_input,resnet_input))
                total = len(bb_list[i*batch_size + img_n])
                total_bbox += total
                count = [0]*total
                for window in sliding_windows:
                    total = len()
                    input_conv = img.crop(window).resize((resnet_input,resnet_input))
                    input_var = torch.autograd.Variable(input_conv, volatile = True).cuda()                
                    output = resnet18(input_var)
                    output = output.float()
                    pred = output.max(1)[1].cpu().data.numpy()
                    for bbox in bb_list[i*batch_size + 1]:
                        ra = Rectangle(bbox[0], bbox[1], bbox[2], bbox[3])
                        rb = Rectangle(window[0], window[1], window[2], window[3])
                        union = (window[3]-window[1])*(window[2]-window[0]) + (bbox[3]- bbox[1])*(bbox[2]- bbox[0])
                        if (area(ra, rb) > 0.5*union and pred == bbox[4]):
                            count[bb_list[i*batch_size + 1].index(bbox)] = 1
                total_correct_bbox += sum(count)
    accuracy = total_correct_bbox / total_bbox *100
    print('Acuuracy: '+str(accuracy))
    

In [140]:
%time daenerys_test(resnet18)

NameError: name 'resnet18' is not defined

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"